# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [230]:
import pandas as pd

In [231]:
df = pd.read_csv('previsao_de_renda.csv')

In [232]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

---

In [233]:
# Carregar os dados do CSV para um DataFrame do Pandas
data = pd.read_csv('previsao_de_renda.csv').drop(columns='Unnamed: 0', axis=1)
data = data.dropna()
data.head()
#data = data.head(50)
#data.to_csv('data.csv')

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [234]:
#data = pd.get_dummies(data)

In [235]:
#dados = data
from scipy import stats
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import patsy

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

In [236]:
# Converter variáveis categóricas em variáveis dummy (one-hot encoding)
data = pd.get_dummies(data, columns=['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'])

# Separar variáveis independentes (covariáveis) e dependente (log da renda)
X = data.drop(columns=['renda', 'data_ref', 'id_cliente'])  # Variáveis independentes
y = np.log(data['renda'])  # Logaritmo natural da renda como variável dependente

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar o modelo de regressão linear aos dados de treinamento
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred = model.predict(X_test)

# Calcular o erro quadrático médio (RMSE) das previsões
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# Coeficientes do modelo
coefficients = pd.DataFrame({'Variável': X.columns, 'Coeficiente': model.coef_})
print("\nCoeficientes:")
print(coefficients)

RMSE: 0.7187697659128964

Coeficientes:
                         Variável  Coeficiente
0                      qtd_filhos    -0.296660
1                           idade     0.005326
2                   tempo_emprego     0.061213
3           qt_pessoas_residencia     0.321038
4                          sexo_F    -0.393640
5                          sexo_M     0.393640
6          posse_de_veiculo_False    -0.018213
7           posse_de_veiculo_True     0.018213
8           posse_de_imovel_False    -0.046086
9            posse_de_imovel_True     0.046086
10         tipo_renda_Assalariado    -0.029529
11            tipo_renda_Bolsista     0.185090
12          tipo_renda_Empresário     0.135327
13         tipo_renda_Pensionista    -0.334019
14    tipo_renda_Servidor público     0.043131
15              educacao_Primário    -0.011860
16         educacao_Pós graduação     0.091390
17            educacao_Secundário    -0.059350
18     educacao_Superior completo     0.051305
19   educacao_Superi

- Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.

In [237]:
# Remover espaços dos nomes das colunas
data.columns = data.columns.str.replace(' ', '_')

# Definir a fórmula para o modelo com Patsy
formula = 'np.log(renda) ~ ' + ' + '.join(data.columns.drop(['renda', 'data_ref', 'id_cliente']))

# Criar a matriz de design com variáveis qualitativas como dummies
y, X = patsy.dmatrices(formula, data=data, return_type='dataframe')

# Dividir os dados em conjuntos de treinamento e teste (80% treinamento, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar um modelo de regressão linear aos dados de treinamento
model = LinearRegression()
model.fit(X_train, y_train)

# Avaliar o desempenho do modelo usando o erro quadrático médio (RMSE) nos dados de teste
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# Coeficientes do modelo
coefficients = pd.DataFrame({'Variável': X_train.columns, 'Coeficiente': model.coef_[0]})
print("\nCoeficientes:")
print(coefficients)

RMSE: 0.7185964275644758

Coeficientes:
                                 Variável   Coeficiente
0                               Intercept -3.370396e+07
1                          sexo_F[T.True]  1.584735e+10
2                          sexo_M[T.True]  1.584735e+10
3          posse_de_veiculo_False[T.True] -2.258010e+11
4           posse_de_veiculo_True[T.True] -2.258010e+11
5           posse_de_imovel_False[T.True] -5.096690e+11
6            posse_de_imovel_True[T.True] -5.096690e+11
7          tipo_renda_Assalariado[T.True] -1.222614e+11
8             tipo_renda_Bolsista[T.True] -1.222614e+11
9           tipo_renda_Empresário[T.True] -1.222614e+11
10         tipo_renda_Pensionista[T.True] -1.222614e+11
11    tipo_renda_Servidor_público[T.True] -1.222614e+11
12              educacao_Primário[T.True] -2.979523e+11
13         educacao_Pós_graduação[T.True] -2.979523e+11
14            educacao_Secundário[T.True] -2.979523e+11
15     educacao_Superior_completo[T.True] -2.979523e+11
16   edu

- Mantenha sempre a categoria mais frequente como casela de referência

- Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:

In [238]:
import statsmodels.api as sm

# Ajustar o modelo inicial
model = sm.OLS(y_train, X_train)
results = model.fit()

# Obter os valores-p
p_values = results.pvalues

# Encontrar a variável menos significativa
least_significant_variable = p_values.idxmax()

# Remover a variável menos significativa
X_train_updated = X_train.drop(least_significant_variable, axis=1)
X_test_updated = X_test.drop(least_significant_variable, axis=1)

# Ajustar o modelo novamente sem a variável menos significativa
model_updated = sm.OLS(y_train, X_train_updated)
results_updated = model_updated.fit()

# Imprimir os resultados atualizados
print(results_updated.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     227.4
Date:                Wed, 10 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:26:58   Log-Likelihood:                -10864.
No. Observations:                9941   AIC:                         2.178e+04
Df Residuals:                    9916   BIC:                         2.196e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

- Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.

Para avaliar se o modelo melhorou ou piorou após remover a variável menos significativa, podemos observar alguns indicadores-chave nos resultados do modelo, como o coeficiente de determinação (R-squared), o erro padrão da regressão (Root Mean Square Error - RMSE), os valores-p e as estatísticas dos coeficientes.

R-squared: O coeficiente de determinação indica a proporção da variabilidade na variável dependente que é explicada pelo modelo. Um R-squared mais próximo de 1 indica um melhor ajuste do modelo aos dados.

RMSE: O erro padrão da regressão indica a dispersão dos resíduos em torno da linha de regressão. Um RMSE menor indica um melhor ajuste do modelo aos dados.

Valores-p: Os valores-p indicam a significância estatística dos coeficientes. Valores-p menores que um nível de significância (geralmente 0,05) indicam que os coeficientes são estatisticamente significativos.

Estatísticas dos coeficientes: Além dos valores-p, também podemos observar as estatísticas dos coeficientes, como os intervalos de confiança e os valores t, para entender a precisão das estimativas dos coeficientes.

Ao comparar esses indicadores antes e depois de remover a variável menos significativa, podemos ter uma ideia se o modelo melhorou ou piorou.

Se você puder fornecer os resultados antes e depois da remoção da variável menos significativa, posso ajudar a analisar e comparar os indicadores para determinar se o modelo melhorou ou piorou.

- Observe os parâmetros e veja se algum se alterou muito.

In [239]:
# Coeficientes antes da remoção
print("Coeficientes antes da remoção:")
print(results.summary())

# Coeficientes depois da remoção
print("\nCoeficientes depois da remoção:")
print(results_updated.summary())

Coeficientes antes da remoção:
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     227.4
Date:                Wed, 10 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:26:58   Log-Likelihood:                -10864.
No. Observations:                9941   AIC:                         2.178e+04
Df Residuals:                    9916   BIC:                         2.196e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 